In [4]:
#  !pip install langdetect
#  !pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import re
import numpy as np
import pickle
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# from langdetect import detect
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping, ModelCheckpoint
from google.colab import files


# Download required NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('universal_tagset')

file_path = '/content/drive/MyDrive/twitter_dataset.csv'
df = pd.read_csv(file_path)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [18]:
df = df.drop(columns=["tweetId","user", "created", "no_of_likes", "source_of_tweet"])

In [19]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Define the text_preprocessing function
def text_preprocessing(text):
    try:
        url_pattern = r"((http://)[^ ]*|(https://)[^ ]*|(www\.)[^ ]*)"
        user_pattern = r"@[^\s]+"
        entity_pattern = r"&.*;"
        neg_contraction = r"n't\W"
        non_alpha = "[^a-z]"
        cleaned_text = text.lower()
        cleaned_text = re.sub(neg_contraction, " not ", cleaned_text)
        cleaned_text = re.sub(url_pattern, " ", cleaned_text)
        cleaned_text = re.sub(user_pattern, " ", cleaned_text)
        cleaned_text = re.sub(entity_pattern, " ", cleaned_text)
        cleaned_text = re.sub(non_alpha, " ", cleaned_text)
        tokens = word_tokenize(cleaned_text)
        # provide POS tag for lemmatization to yield better result
        word_tag_tuples = pos_tag(tokens, tagset="universal")
        tag_dict = {"NOUN": "n", "VERB": "v", "ADJ": "a", "ADV": "r"}
        final_tokens = []
        for word, tag in word_tag_tuples:
            if len(word) > 1 and word not in stopwords:
                if tag in tag_dict:
                    final_tokens.append(lemmatizer.lemmatize(word, tag_dict[tag]))
                else:
                    final_tokens.append(lemmatizer.lemmatize(word))
        return " ".join(final_tokens)
    except:
        return np.nan

In [29]:
# Apply text preprocessing to create a new column
df["cleaned_sentiment"] = df["sentiment"].apply(text_preprocessing)

# Splitting the filtered DataFrame into train and test datasets
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(index=train_df.index)

In [30]:
# Drop rows with NaN values
df_dropped = df.dropna()

# Drop unnamed columns
df_dropped = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Print the modified DataFrame
print(df_dropped)

# Save the modified DataFrame to a CSV file
df_dropped.to_csv('modified_data.csv', index=False)

# Download the CSV file
files.download('modified_data.csv')

                                              sentiment  \
0     No, Pfizer has not admitted to lying about tes...   
1     @Pitahkun @amerix All vaccines provided in the...   
2     @kaburifrancis @paulinenjoroge All vaccines pr...   
3     Levels of handwashing and vaccine uptake in Ke...   
4     Influence of Government Policies on Hand Washi...   
...                                                 ...   
3265  Kenya being the leading safari destination has...   
3266  @USAID @WHO is hiring a short-term consultant ...   
3267  #Kenya: Kenya set to receive COVID-19 vaccines...   
3268  #Kenya: Kenya set to receive COVID-19 vaccines...   
3269  #Kenya: COVID 19 Vaccine: Healthcare workers w...   

                                      cleaned_sentiment  
0     pfizer admit lie test effectiveness covid vacc...  
1     vaccine provide country thoroughly test prove ...  
2     vaccine provide country thoroughly test prove ...  
3     level handwashing vaccine uptake kenya uganda ...  
4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# df = df[df["cleaned_sentiment"].notna() & (df["cleaned_sentiment"] != "")]

# train_df = df.sample(frac=0.8, random_state=42)
# test_df = df.drop(index=train_df.index)

In [17]:
# # Filtering out rows with non-empty and non-null "cleaned_tweet" column
# df = df[df["cleaned_sentiment"].notna() & (df["cleaned_sentiment"] != "")]

# # Splitting the filtered DataFrame into train and test datasets
# train_df = df.sample(frac=0.8, random_state=42)
# test_df = df.drop(index=train_df.index)

# # Printing the resulting DataFrames
# print("Train DataFrame:")
# print(train_df.head())  # Display first few rows of the train_df

# print("\nTest DataFrame:")
# print(test_df.head())  # Display first few rows of the test_df

In [23]:
X = train_df.cleaned_sentiment
y = train_df.sentiment
X_train, X_val, y_train, y_val = train_test_split(
    X, y, train_size=0.8, random_state=42
)

In [24]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)    
vocab_size = len(tokenizer.word_index) + 1
print("Number of Unique Words: {}".format(vocab_size))

# saving tokenizer
with open("/content/drive/MyDrive/static/tokenizer.pickle", "wb") as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Number of Unique Words: 4626


In [25]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train))
input_len = X_train.shape[1]
X_val = pad_sequences(tokenizer.texts_to_sequences(X_val), maxlen=input_len)
y_train = y_train.replace({"Negative": 0, "Positive": 1})
y_val = y_val.replace({"Negative": 0, "Positive": 1})

In [26]:
# # download the pre-trained GloVe embedding (27B tokens based on 2B tweets)
# file_url = "https://nlp.stanford.edu/data/glove.twitter.27B.zip"    
# !wget $file_url
# !unzip /content/glove.twitter.27B.zip

# load the pre-trained GloVe embedding with 200 dimensions into a dictionary
embeddings_index = {}
with open("/content/drive/MyDrive/glove.twitter.27B.200d.txt", "r") as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# create our embedding matrix
embed_dim = 200
embedding_matrix = np.zeros((vocab_size, embed_dim))
for word, index in tokenizer.word_index.items():
    if word in embeddings_index:
        embedding_matrix[index] = embeddings_index[word]

# create the embedding layer
embedding_layer = Embedding(input_dim=vocab_size, 
                            output_dim=embed_dim, 
                            weights=[embedding_matrix], 
                            input_length=input_len, 
                            trainable=False)

In [27]:
model = Sequential()
model.add(embedding_layer)
model.add(SpatialDropout1D(0.4))
model.add(LSTM(150, dropout=0.25, recurrent_dropout=0.25))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 28, 200)           925200    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 28, 200)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 150)               210600    
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 1,135,951
Trainable params: 210,751
Non-trainable params: 925,200
_________________________________________________________________
None


In [28]:
# Assuming you have a cleaned DataFrame called 'cleaned_sentiment'
cleaned_sentiment = pd.DataFrame(sentiment)  # Replace ... with your data

# Save the DataFrame as a CSV file
cleaned_sentiment.to_csv('cleaned_sentiment.csv', index=False)

# Download the file
files.download('cleaned_sentiment.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [65]:
# es_callback = EarlyStopping(monitor="val_loss", patience=3)
# mc_callback = ModelCheckpoint(
#     filepath="/content/drive/MyDrive/static/lstm_model-{epoch:02d}.h5",
#     monitor="val_loss",
#     mode="min",
#     save_best_only=True,
# )

# history = model.fit(
#     X_train,
#     y_train,
#     epochs=10,
#     validation_data=(X_val, y_val),
#     callbacks=[es_callback, mc_callback],
#     batch_size=32
# )

In [48]:
# model = load_model("/content/drive/MyDrive/static/lstm_model-08.h5")
# sequences = pad_sequences(
#     tokenizer.texts_to_sequences(test_df["cleaned_tweet"]), maxlen=input_len
# )
# test_df["score"] = model.predict(sequences)
# test_df["pred_sentiment"] = test_df["score"].apply(
#     lambda x: "Positive" if x >= 0.50 else "Negative"
# )

In [50]:
# accuracy_score(test_df["sentiment"], test_df["pred_sentiment"])

In [51]:
# tn, fp, fn, tp = confusion_matrix(
#     test_df["sentiment"], test_df["pred_sentiment"]
# ).ravel()
# tnr = tn / (tn + fp)
# tpr = tp / (tp + fn)
# print("True Negative Rate: {:.3f}".format(tnr))
# print("True Positive Rate: {:.3f}".format(tpr))

KeyError: ignored